In [1]:
# https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed
# Import Libraries
import os
import tweepy
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import csv
import time
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation 

In [2]:
# Authentication Keys
# https://developer.twitter.com/en/portal/projects/1268606526274580480/apps/18064440/keys

# Think of these as the user name and password that represents
# your Twitter developer app when making API requests. 
consumerKey = '9gjPeGkMZj2935QAVEGEFqZG2'
consumerSecret = 'jSJbncxF8OifGepXcoR5uadfvhlVxXgoO9P5zDtMzNQidimRYx'

# User-specific credentials used to authenticate OAuth 1.0a API requests. 
# They specify the Twitter account the request is made on behalf of.
accessToken = '1130514658673213441-4GKVmzpYX67W2HlNiccXVIR3e0kzi7'
accessTokenSecret = 'HBN0T2QDg2nOcFOlZP0GyS9MwYGkjb3gtsrE7ebxKnJn8'

In [3]:
# Create the authentication object
auth = tweepy.OAuthHandler(consumerKey, consumerSecret) 
    
# Set the access token and access token secret
auth.set_access_token(accessToken, accessTokenSecret) 

# Creating the API object while passing in auth information
api = tweepy.API(auth)

In [4]:
# Constants 
START_DATE = '2020-05-01'
TWEET_NUMBER = 100
SEARCH_WORD = '#CRA'
RATE_LIMIT = 180
SLEEP_TIME = 900/180 # 15 minutes = 900 seconds

In [5]:
# Collect tweets using Cursor method
# http://docs.tweepy.org/en/v3.5.0/cursor_tutorial.html
def buildTestSet():
    tweet_list = []
    tweets_fetched = tweepy.Cursor(api.search,
                  q=SEARCH_WORD,
                  lang='en',
                  since=START_DATE).items(5) 

    for tweet in tweets_fetched:
        tweet_list.append(tweet.text)
        # print(tweet.text)
        
    # Array where Test Set is stored
    return tweet_list

In [6]:
# Build the test set
testDataSet = buildTestSet()

In [7]:
# data https://github.com/karanluthra/twitter-sentiment-training/blob/master/corpus.csv
# https://www.hugedomains.com/domain_profile.cfm?d=sananalytics&e=com
# https://github.com/shiyanboxer/twitter-sentiment-training
# For this task, we will be using the amazing Niek Sanders’ Corpus of over 5000 hand-classified tweets, which makes it quite reliable. There’s also a catch here. Twitter does not allow storing tweets on a personal device, even though all such data is publicly available. Therefore, the corpus includes a keyword (topic of the tweet), a label (pos/neg) and a tweet ID number for every tweet (i.e. row in our CSV corpus). You can get the file containing the corpus from the original site, or through this link of a personal repository.

def buidTrainingSet(corpusFile, tweetDataFile):
    corpus = []    
    with open(corpusFile,'rb') as csvfile:
        lineReader = csv.reader(csvfile,delimiter=',', quotechar="\"")
        for row in lineReader:
            corpus.append({"tweet_id":row[2], "label":row[1], "topic":row[0]})
            
    trainingDataSet = []

    for tweet in corpus:
        try:
            status = api.GetStatus(tweet["tweet_id"])
            print("Tweet fetched" + status.text)
            tweet["text"] = status.text
            trainingDataSet.append(tweet)
            time.sleep(SLEEP_TIME) 
        except: 
            continue
    # now we write them to the empty CSV file
    with open(tweetDataFile,'wb') as csvfile:
        linewriter = csv.writer(csvfile,delimiter=',',quotechar="\"")
        for tweet in trainingDataSet:
            try:
                linewriter.writerow([tweet["tweet_id"], tweet["text"], tweet["label"], tweet["topic"]])
            except Exception as e:
                print(e)
    return trainingDataSet

In [8]:
# corpusFile is the string path to the Niek Sanders’ CSV corpus file we downloaded. This file, as mentioned earlier, includes the tweet’s topic, label and id.
# tweetDataFile is the string path to the file we would like to save the full tweets in. In contrast to corpusFile, this file will include every tweet’s text as well as topic, label and id.

corpusFile = "./Documents/twitter-sentiment-training-master/twitter-sentiment-training-master/corpus.csv"
tweetDataFile = "./Documents/twitter-sentiment-training-master/twitter-sentiment-training-master/tweetDataFile.csv"

trainingData = buildTrainingSet(corpusFile, tweetDataFile)

NameError: name 'buildTrainingSet' is not defined

In [ ]:
class PreProcessTweets:
    def __init__(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['AT_USER','URL'])
    # The processTweets function just loops through all the tweets input into it, calling its neighboring function processTweet on every tweet in the list    
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = word_tokenize(tweet) # remove repeated characters (helloooooooo into hello)
        return [word for word in tweet if word not in self._stopwords]
    
# First, we will create a variable that refers to it (an object), and then call it on both the Training and Test sets as we discussed earlier:    
tweetProcessor = PreProcessTweets()
preprocessedTrainingSet = tweetProcessor.processTweets(trainingData)
preprocessedTestSet = tweetProcessor.processTweets(testDataSet)

"""
The latter does the actual pre-processing by first making all the text in lower-case letters. This is merely because, in almost all programming languages, “cAr” is not interpreted the same way as “car”. Therefore, it is better to normalize all characters to be lower-case across all our data. Secondly, URLs and usernames are removed from the tweet. This is for the reasons we disclosed earlier in the article. Afterwards, the number sign (i.e. #) is removed from every hashtag, in order to avoid hashtags being processed differently. Last but not least, duplicate characters are rid off of, in order to ensure that no important word goes unprocessed even if it is spelled out in an unusual way (e.g. “caaaaar” becomes “car”). Finally, the tweet’s text is broken into words (tokenized) in order to ease its processing in the upcoming stages.
"""

In [ ]:
"""
This is just creating a list of all_words we have in the Training set, breaking it into word features. Those word_features are basically a list of distinct words, each of which has its frequency (number of occurrences in the set) as a key.
"""
def buildVocabulary(preprocessedTrainingData):
    all_words = []
    
    for (words, sentiment) in preprocessedTrainingData:
        all_words.extend(words)

    wordlist = nltk.FreqDist(all_words)
    word_features = wordlist.keys()
    
    return word_features

In [9]:
"""
comparing every word against the tweet at hand, associating a number with the word with true or false
"""
def extract_features(tweet):
    tweet_words=set(tweet)
    features={}
    for word in word_features:
        features['contains(%s)' % word]=(word in tweet_words)
    return features 

In [10]:
# Now we can extract the features and train the classifier
"""
Let’s now call the last two functions we have written. This will build our final feature vector, with which we can proceed on to training.
The NTLK built-in function apply_features does the actual feature extraction from our lists. Our final feature vector is trainingFeatures.
"""
word_features = buildVocabulary(preprocessedTrainingSet)
trainingFeatures=nltk.classify.apply_features(extract_features,preprocessedTrainingSet)

NameError: name 'buildVocabulary' is not defined

In [11]:
# Training the classifier
# Thanks to NLTK, it will only take us a function call to train the model as a Naive Bayes Classifier, since the latter is built into the library:
NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)

NameError: name 'trainingFeatures' is not defined

In [12]:
NBResultLabels = [NBayesClassifier.classify(extract_features(tweet[0])) for tweet in preprocessedTestSet]

NameError: name 'preprocessedTestSet' is not defined

In [13]:
# get the majority vote
if NBResultLabels.count('positive') > NBResultLabels.count('negative'):
    print("Overall Positive Sentiment")
    print("Positive Sentiment Percentage = " + str(100*NBResultLabels.count('positive')/len(NBResultLabels)) + "%")
else: 
    print("Overall Negative Sentiment")
    print("Negative Sentiment Percentage = " + str(100*NBResultLabels.count('negative')/len(NBResultLabels)) + "%")

NameError: name 'NBResultLabels' is not defined